## 자치구 별 교육 기관 데이터

In [1]:
import pandas as pd
import mysql.connector

### 어린이집 데이터 뽑아오기

In [2]:
daycare_center = pd.read_csv('/home/lee/dev_ws/projects/project01/data/어린이집.csv', encoding='euc-kr')
daycare_center.head()

,통계연도,자치구코드,자치구명,시설수합계,어린이집수_국공립,어린이집수_사회복지법인,어린이집수_법인단체등,어린이집수_민간,어린이집수_가정,어린이집수_부모협동,어린이집수_직장
0,2024,11410,서대문,126,63,0,2,26,29,1,5
1,2024,11710,송파,296,108,1,3,68,95,0,21
2,2024,11680,강남,174,70,0,0,33,31,1,39
3,2024,11650,서초,157,94,0,1,18,22,2,20
4,2024,11620,관악,158,79,1,3,33,37,0,5


In [3]:
daycare_center = daycare_center.iloc[0:25]
daycare_center = daycare_center[['자치구명', '시설수합계']]
daycare_center.rename(columns={'자치구명':'자치구',
                                        '시설수합계':'계'}, inplace=True)

In [4]:
# 25개 자치구 확인하기
len(daycare_center.index.unique())

25

In [5]:
daycare_center.reset_index()
daycare_center.head()

,자치구,계
0,서대문,126
1,송파,296
2,강남,174
3,서초,157
4,관악,158


### 유치원 데이터 뽑아오기

In [6]:
preschool = pd.read_csv('/home/lee/dev_ws/projects/project01/data/유치원.csv', header=2,encoding='utf-8')
preschool.head()

,자치구별(1),자치구별(2),소계,소계.1,소계.2,남자,여자,소계.3,남자.1,여자.1,...,남자.3,여자.3,소계.6,남자.4,여자.4,소계.7,일반교실,교과교실,특별교실,수준별교실
0,합계,소계,754,3576,61128,31361,29767,6425,94,6331,...,13423,12610,23952,12100,11852,3212,2985,34,177,16
1,합계,종로구,16,50,904,471,433,89,2,87,...,190,173,361,175,186,49,45,2,2,-
2,합계,중구,13,53,864,426,438,93,1,92,...,167,168,302,154,148,59,54,-,4,1
3,합계,용산구,13,66,1242,660,582,135,2,133,...,251,268,432,226,206,68,59,4,5,-
4,합계,성동구,28,107,1574,832,742,186,2,184,...,396,335,620,322,298,85,80,1,4,-


In [7]:
preschool = preschool[['자치구별(2)','소계']]
preschool.rename(columns={'자치구별(2)':'자치구', '소계':'계'}, inplace=True)
preschool = preschool[preschool['자치구']!='소계']

In [9]:
preschool = preschool.set_index('자치구')
preschool = preschool.reset_index()
preschool.head()

,자치구,계
0,종로구,16
1,중구,13
2,용산구,13
3,성동구,28
4,광진구,24


### 초등학교 데이터 가져오기

In [10]:
elementary = pd.read_csv('/home/lee/dev_ws/projects/project01/data/초등학교.csv',header=3, encoding='utf-8')
elementary = elementary[['자치구별(2)','소계']]
elementary.rename(columns={'자치구별(2)':'자치구', '소계':'계'}, inplace=True)
elementary = elementary[elementary['자치구']!='소계']

In [11]:
elementary = elementary.set_index('자치구')
elementary = elementary.reset_index()
elementary.head()

,자치구,계
0,종로구,13
1,중구,12
2,용산구,15
3,성동구,21
4,광진구,21


### 중학교 데이터 가져오기

In [12]:
middle = pd.read_csv('/home/lee/dev_ws/projects/project01/data/중학교_390.csv', encoding='euc-kr')

middle = middle[['도로명주소', '학교명', '설립구분']]
middle.head()



,도로명주소,학교명,설립구분
0,서울특별시 성동구 왕십리로 189,행당중학교,공립
1,서울특별시 성동구 마조로 42,한양대학교사범대학부속중학교,사립
2,서울특별시 광진구 뚝섬로41길 33,자양중학교,공립
3,서울특별시 광진구 용마산로22길 76,용곡중학교,공립
4,서울특별시 성동구 한림말길 11,옥정중학교,공립


In [13]:
middle['자치구'] = [gu.split(' ')[1] for gu in middle['도로명주소']]
middle.head()

,도로명주소,학교명,설립구분,자치구
0,서울특별시 성동구 왕십리로 189,행당중학교,공립,성동구
1,서울특별시 성동구 마조로 42,한양대학교사범대학부속중학교,사립,성동구
2,서울특별시 광진구 뚝섬로41길 33,자양중학교,공립,광진구
3,서울특별시 광진구 용마산로22길 76,용곡중학교,공립,광진구
4,서울특별시 성동구 한림말길 11,옥정중학교,공립,성동구


In [14]:
middle = pd.pivot_table(
    data=middle,
    columns='자치구',
    values='학교명',
    aggfunc='count'
).T.reset_index()

middle.rename(columns={'학교명':'계'},inplace=True)

middle.head()

,자치구,계
0,강남구,24
1,강동구,19
2,강북구,13
3,강서구,22
4,관악구,16


### 고등학교 데이터 가져오기

In [15]:
high = pd.read_csv('/home/lee/dev_ws/projects/project01/data/고등학교_318.csv', encoding='euc-kr')
high.head()

,학교종류명,설립구분,표준학교코드,학교명,영문학교명,관할조직명,도로명우편번호,도로명주소,도로명상세주소,전화번호,...,고등학교일반실업구분명,특수목적고등학교계열명,입시전후기구분명,주야구분명,설립일자,개교기념일,시도교육청코드,시도교육청명,소재지명,적재일시
0,고등학교,공립,7011575,서울반도체고등학교,Seoul Semiconductor high school,서울특별시교육청,2511,서울특별시 동대문구 겸재로 21,", 서울반도체고등학교 (휘경동)",02-2230-9885,...,전문계,NaN,전기,주간,19971220,19980702,B10,서울특별시교육청,서울특별시,20230615
1,고등학교,사립,7011569,미림마이스터고등학교,Mirim Meister High School,서울특별시교육청,8821,서울특별시 관악구 호암로 546,미림마이스터고등학교,02-872-4071,...,전문계,산업수요 맞춤형 고등학교,전기,주간,19910301,19910531,B10,서울특별시교육청,서울특별시,20240310
2,고등학교,사립,7011568,염광메디텍고등학교,Yumkwang Meditech High School,서울특별시교육청,1874,서울특별시 노원구 월계로45가길 9,", 염광메디텍고등학교 (월계동)",02-994-3105,...,전문계,NaN,전기,주간,19680301,19680301,B10,서울특별시교육청,서울특별시,20240303
3,고등학교,공립,7011558,서울웹툰애니메이션고등학교,Seoul Webtoon Animation High School,서울특별시교육청,6761,서울특별시 서초구 과천대로 852-23,(방배동),02-6011-4100,...,전문계,NaN,전기,주간,19940105,19940506,B10,서울특별시교육청,서울특별시,20230615
4,고등학교,공립,7011540,솔샘고등학교,Solsaem High School,서울특별시교육청,1190,서울특별시 강북구 인수봉로 39,/ 솔샘고등학교,02-982-6104,...,일반계,NaN,후기,주간,20090301,20090610,B10,서울특별시교육청,서울특별시,20230615


In [16]:
high = high[['도로명주소', '학교명', '설립구분']]
high['자치구'] = [gu.split(' ')[1] for gu in high['도로명주소']]
high = pd.pivot_table(
    data=high,
    columns='자치구',
    values='학교명',
    aggfunc='count'
).T.reset_index()

high.rename(columns={'학교명':'계'},inplace=True)

high.head()


,자치구,계
0,강남구,22
1,강동구,14
2,강북구,7
3,강서구,23
4,관악구,17


### 대학교 데이터 가져오기

In [17]:
university = pd.read_excel('/home/lee/dev_ws/projects/project01/data/대학교.xlsx',header=1)
university.head()

/home/lee/venv/eda_venv/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,시도,행정구,학교수,전문대학,교육대학,일반대학,방송통신대학,산업대학,기술대학,각종학교,원격및사이버대학,사내대학,전공대학,기능대학,대학원,부설대학원
0,전체,계,429.0,136.0,10.0,191.0,1.0,2.0,1.0,2.0,21.0,8.0,3.0,9.0,45.0,"(1,124)"
1,서울,종로구,8.0,1.0,NaN,2.0,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,2.0,(29)
2,서울,중구,3.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0.0,(11)
3,서울,용산구,3.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,(10)
4,서울,성동구,3.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,(21)


In [18]:
university = university[university['시도']=='서울']
university = university[['행정구', '학교수']]
university = university.drop(26)
university.rename(columns={'행정구':'자치구','학교수':'계'},inplace=True)
university.head()

,자치구,계
1,종로구,8.0
2,중구,3.0
3,용산구,3.0
4,성동구,3.0
5,광진구,4.0


### 학원과 독서실 데이터 가져오기

In [19]:
academy_studyspace = pd.read_csv('/home/lee/dev_ws/projects/project01/data/학원_독서실.csv',encoding='utf-8')
academy_studyspace.head()

,자치구별(1),자치구별(2),2023,2023.1,2023.2,2023.3,2023.4,2023.5,2023.6,2023.7,...,2023.11,2023.12,2023.13,2023.14,2023.15,2023.16,2023.17,2023.18,2023.19,2023.20
0,자치구별(1),자치구별(2),사설학원,사설학원,사설학원,사설학원,사설학원,사설학원,사설학원,사설학원,...,사설학원,사설학원,사설학원,사설학원,사설학원,사설학원,사설학원,사설학원,사설학원,사설학원
1,자치구별(1),자치구별(2),소계,학원수 (개),학원수 (개),학원수 (개),학원수 (개),학원수 (개),학원수 (개),학원수 (개),...,학원수 (개),학원수 (개),학원수 (개),학원수 (개),학원수 (개),임시수용능력인원 (명),강사수 (명),강사수 (명),강사수 (명),교수학습공간 (개)
2,자치구별(1),자치구별(2),소계,학교교과 교습학원,학교교과 교습학원,학교교과 교습학원,학교교과 교습학원,학교교과 교습학원,학교교과 교습학원,학교교과 교습학원,...,평생직업 교육학원,평생직업 교육학원,평생직업 교육학원,평생직업 교육학원,평생직업 교육학원,소계,소계,남자,여자,소계
3,자치구별(1),자치구별(2),소계,소계,입시검정 및 보습,국제화,예능,특수교육,전문입시컨설팅,종합,...,직업기술,국제화,인문사회,기예,종합,소계,소계,소계,소계,소계
4,합계,소계,14688,12581,8180,827,2557,1,92,389,...,856,191,236,719,105,1416566,99819,43242,56577,80814


In [20]:
academy_studyspace = academy_studyspace[['자치구별(2)','2023.1','2023.2','2023.4','2023.6','2023.20']]
academy_studyspace

,자치구별(2),2023.1,2023.2,2023.4,2023.6,2023.20
0,자치구별(2),사설학원,사설학원,사설학원,사설학원,사설학원
1,자치구별(2),학원수 (개),학원수 (개),학원수 (개),학원수 (개),교수학습공간 (개)
2,자치구별(2),학교교과 교습학원,학교교과 교습학원,학교교과 교습학원,학교교과 교습학원,소계
3,자치구별(2),소계,입시검정 및 보습,예능,전문입시컨설팅,소계
4,소계,12581,8180,2557,92,80814
5,종로구,100,63,19,-,1085
6,중구,70,38,15,-,675
7,용산구,124,62,31,-,816
8,성동구,265,158,65,-,1826
9,광진구,378,255,83,-,2392


In [21]:
academy_studyspace.rename(columns={
    '자치구별(2)':'자치구',
    '2023.1':'계',
    '2023.2':'입시학원계',
    '2023.4':'예체능학원계',
    '2023.6':'입시컨설팅계',
    '2023.20':'독서실계'
},inplace=True)

academy_studyspace = academy_studyspace.drop([0,1,2,3,4])

academy_studyspace = academy_studyspace.set_index('자치구')


In [22]:
academy_studyspace = academy_studyspace.reset_index()
academy_studyspace

,자치구,계,입시학원계,예체능학원계,입시컨설팅계,독서실계
0,종로구,100,63,19,-,1085
1,중구,70,38,15,-,675
2,용산구,124,62,31,-,816
3,성동구,265,158,65,-,1826
4,광진구,378,255,83,-,2392
5,동대문구,359,231,95,-,2491
6,중랑구,258,174,55,-,1733
7,성북구,447,308,106,-,2566
8,강북구,199,146,42,-,1208
9,도봉구,293,215,62,-,1663


### 학원 데이터

In [24]:
# 입시학원
test_prep_acad = academy_studyspace[['자치구', '입시학원계']]
test_prep_acad.head()

,자치구,입시학원계
0,종로구,63
1,중구,38
2,용산구,62
3,성동구,158
4,광진구,255


In [25]:
# 에체능학원
art_sport_acad = academy_studyspace[['자치구', '예체능학원계']]
art_sport_acad.head()

,자치구,예체능학원계
0,종로구,19
1,중구,15
2,용산구,31
3,성동구,65
4,광진구,83


### 독서실 데이터

In [26]:
study_space = academy_studyspace[['자치구', '독서실계']]
study_space.head()

,자치구,독서실계
0,종로구,1085
1,중구,675
2,용산구,816
3,성동구,1826
4,광진구,2392


### 도서관 데이터 가져오기

In [27]:
library = pd.read_csv('/home/lee/dev_ws/projects/project01/data/도서관.csv',header=2,encoding='utf-8')
library.head()

,자치구별(1),자치구별(2),소계,국립도서관,공공도서관,대학도서관,전문도서관
0,합계,소계,478,3,207,88,180
1,합계,종로구,41,-,9,7,25
2,합계,중구,38,-,9,2,27
3,합계,용산구,20,-,4,3,13
4,합계,성동구,10,-,7,2,1


In [28]:
library = library.drop('자치구별(1)',axis=1)
library = library.drop([0,26])

In [29]:
library.head()

,자치구별(2),소계,국립도서관,공공도서관,대학도서관,전문도서관
1,종로구,41,-,9,7,25
2,중구,38,-,9,2,27
3,용산구,20,-,4,3,13
4,성동구,10,-,7,2,1
5,광진구,13,-,7,3,3


In [30]:
library.rename(columns={
    '자치구별(2)':'자치구',
    '소계':'계'
},inplace=True)

library.head()

,자치구,계,국립도서관,공공도서관,대학도서관,전문도서관
1,종로구,41,-,9,7,25
2,중구,38,-,9,2,27
3,용산구,20,-,4,3,13
4,성동구,10,-,7,2,1
5,광진구,13,-,7,3,3


In [31]:
library = library.set_index('자치구')
library = library.reset_index()
library = library[['자치구','계']]
library.head()

,자치구,계
0,종로구,41
1,중구,38
2,용산구,20
3,성동구,10
4,광진구,13
